In [17]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 1.1 MB/s eta 0:00:0000:0100:01m


In [39]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [118]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.io import read_image
import pandas as pd
import glob
import os

In [119]:
# 데이터 경로 설정
data_dir = "/home/downtown/new_folder/deep-learning/module/module-1/chest_xray"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# 데이터프레임 생성 함수 정의
def create_dataframe(data_dir):
    image_paths = glob.glob(f"{data_dir}/*/*")
    data = {'image_path': [], 'label': []}
    for path in image_paths:
        if 'NORMAL' in path:
            data['image_path'].append(path)
            data['label'].append(0)  # NORMAL -> 0
        elif 'PNEUMONIA' in path:
            data['image_path'].append(path)
            data['label'].append(1)  # PNEUMONIA -> 1
    return pd.DataFrame(data)

# Train과 Test 데이터프레임 생성
train_df = create_dataframe(train_dir)
test_df = create_dataframe(test_dir)

In [120]:
# 이미지 파일 경로 확인
print("Train 이미지 파일 수:", len(glob.glob(f"{train_dir}/*/*")))
print("Test 이미지 파일 수:", len(glob.glob(f"{test_dir}/*/*")))

Train 이미지 파일 수: 5216
Test 이미지 파일 수: 640


In [121]:
# Dataset 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        image = read_image(img_path)  # 이미지 로드 (0-255 사이의 값, uint8 형식)
        label = self.df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)  # 변환 적용

        return image, label

In [122]:
# Dataset 인스턴스 생성
dataset = ImageDataset(train_df)

# 첫 번째 이미지의 텐서 shape 읽기
image_tensor, _ = dataset[0]  # 데이터셋의 첫 번째 항목을 불러옴
print(f"첫 번째 이미지의 텐서 shape: {image_tensor.shape}")


첫 번째 이미지의 텐서 shape: torch.Size([1, 1233, 1596])


In [123]:
# 이미지 전처리 및 데이터 로더 생성
transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.Grayscale(num_output_channels=1),
    transforms.ConvertImageDtype(torch.float32),  
    transforms.Normalize(mean=[0.5], std=[0.5])  # 흑백 이미지의 평균과 표준편차로 정규화
])

# 전체 학습 데이터셋 생성
full_train_dataset = ImageDataset(train_df, transform=transform)

# 학습 데이터셋과 검증 데이터셋으로 나누기 (80% train, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# 테스트 데이터셋 생성
test_dataset = ImageDataset(test_df, transform=transform)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [124]:
# 이진 분류 모델 정의
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layer_1 = nn.Linear(256 * 256, 128) 
        self.bn1 = nn.BatchNorm1d(128)  
        self.layer_2 = nn.Linear(128, 64) 
        self.bn2 = nn.BatchNorm1d(64) 
        self.layer_3 = nn.Linear(64, 1) 

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layer_1(x)
        x = self.bn1(x) 
        x = torch.relu(x)  
        x = self.layer_2(x)
        x = self.bn2(x)  
        x = torch.relu(x)  
        z = self.layer_3(x)
        return z

model = BinaryClassificationModel()

# 손실 함수 및 옵티마이저 설정
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습 루프
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels in train_loader:
        labels = labels.float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)  # labels 모양을 (batch, 1)로 맞춤
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    # 검증 단계
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}, Train Loss: {epoch_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}")


Epoch 1, Train Loss: 0.20298243063779278, Validation Loss: 0.3269733734654658
Epoch 2, Train Loss: 0.13299307006745392, Validation Loss: 0.12622957123499928
Epoch 3, Train Loss: 0.10403445456887929, Validation Loss: 0.09919128945153771
Epoch 4, Train Loss: 0.09094876746597294, Validation Loss: 0.09144820820427302
Epoch 5, Train Loss: 0.07405285625398614, Validation Loss: 0.10049939130178907
Epoch 6, Train Loss: 0.06811378601554582, Validation Loss: 0.10263834443564217
Epoch 7, Train Loss: 0.05098296432108938, Validation Loss: 0.12088930544753869
Epoch 8, Train Loss: 0.05414596599800413, Validation Loss: 0.16770018290051006
Epoch 9, Train Loss: 0.048860807483660354, Validation Loss: 0.10821807542533586
Epoch 10, Train Loss: 0.039668656118704675, Validation Loss: 0.12876197989945384
Epoch 11, Train Loss: 0.03124942039566129, Validation Loss: 0.12849632738101663
Epoch 12, Train Loss: 0.03900818893142073, Validation Loss: 0.15138642003787964
Epoch 13, Train Loss: 0.039618083264079756, Vali

In [127]:
# 모델 평가 및 예측
model.eval()
test_loss = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.float().unsqueeze(1)
        outputs = model(images)
        loss = loss_function(outputs, labels)  # 테스트 손실 계산
        test_loss += loss.item()
        
        predicted = (torch.sigmoid(outputs) > 0.5).float()  # 0.5 기준으로 이진 분류
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy().flatten())

# 최종 테스트 손실 출력
print(f"Test Loss: {test_loss / len(test_loader)}")

Test Loss: 2.1062989481584737


In [128]:
# 분류 레포트 출력

from sklearn.metrics import classification_report
report = classification_report(all_labels, all_predictions, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

      NORMAL       0.95      0.35      0.51       242
   PNEUMONIA       0.71      0.99      0.83       398

    accuracy                           0.75       640
   macro avg       0.83      0.67      0.67       640
weighted avg       0.80      0.75      0.71       640

